# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Notebook
- Build a dataframe of neighborhoods in Vadodara, Gujarat by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new shopping mall
***
### 1. Import libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [3]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Urban_and_suburban_areas_of_Vadodara").text

In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [5]:
# create a list to store neighborhood data
neighborhoodList = []

In [6]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [7]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Akota
1,Alkapuri
2,Fatehgunj
3,Karelibaug
4,Kayavarohan


In [8]:
# print the number of rows of the dataframe
kl_df.shape

(14, 1)

### 3. Get the geographical coordinates

In [9]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Vadodara, Gujarat'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [11]:
coords

[[22.29128000000003, 73.16947000000005],
 [22.314190000000053, 73.17424000000005],
 [22.325480000000027, 73.18949000000003],
 [22.32567000000006, 73.20176000000004],
 [22.074140000000057, 73.25027000000006],
 [22.27547000000004, 73.19164000000006],
 [22.312340000000063, 73.16282000000007],
 [22.30394000000007, 73.19981000000007],
 [22.415660898555597, 73.12300807514401],
 [22.31281000000007, 73.18767000000008],
 [22.32367000000005, 73.16472000000005],
 [22.28469000000007, 73.14562000000006],
 [22.312620000000038, 73.38860000000005],
 [22.30946000000006, 73.17993000000007]]

In [12]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [13]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [14]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(14, 3)


,Neighborhood,Latitude,Longitude
0,Akota,22.291280,73.169470
1,Alkapuri,22.314190,73.174240
2,Fatehgunj,22.325480,73.189490
3,Karelibaug,22.325670,73.201760
4,Kayavarohan,22.074140,73.250270
5,Manjalpur,22.275470,73.191640
6,"Race Course, Vadodara",22.312340,73.162820
7,Raopura,22.303940,73.199810
8,Sankheda,22.415661,73.123008
9,Sayajigunj,22.312810,73.187670


In [15]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

### 4. Create a map of Vadodara with neighborhoods superimposed on top

In [49]:
# get the coordinates of Vadodara
address = 'Vadodara, Gujarat'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vadodara, Gujarat {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vadodara, Gujarat 22.2973142, 73.1942567.


In [50]:

map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [20]:
# save the map as HTML file
map_kl.save('map_kl.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [27]:
# define Foursquare Credentials and Version
CLIENT_ID = 'LPNUPRJ123HHBQEMUTXCZF4VAWQQZLWZW0MXIM0YXCHONEBU' # your Foursquare ID
CLIENT_SECRET = 'Y3E2ALJ13OGDHGYQKH2405ILFEVQ2MK24YVUG5HCPD3IVNDZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LPNUPRJ123HHBQEMUTXCZF4VAWQQZLWZW0MXIM0YXCHONEBU
CLIENT_SECRET:Y3E2ALJ13OGDHGYQKH2405ILFEVQ2MK24YVUG5HCPD3IVNDZ


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [28]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [29]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(506, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Akota,22.29128,73.16947,Dairy Den,22.296099,73.163797,Ice Cream Shop
1,Akota,22.29128,73.16947,Pizza on the Rock,22.298322,73.164806,Pizza Place
2,Akota,22.29128,73.16947,Kai Asia,22.293349,73.174587,Asian Restaurant
3,Akota,22.29128,73.16947,Havmor Restaurant,22.301754,73.165713,Indian Restaurant
4,Akota,22.29128,73.16947,McDonald's,22.297058,73.164798,Fast Food Restaurant


**Let's check how many venues were returned for each neighorhood**

In [30]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Akota,40,40,40,40,40,40
Alkapuri,92,92,92,92,92,92
Fatehgunj,37,37,37,37,37,37
Karelibaug,31,31,31,31,31,31
Manjalpur,9,9,9,9,9,9
"Race Course, Vadodara",73,73,73,73,73,73
Raopura,24,24,24,24,24,24
Sayajigunj,67,67,67,67,67,67
Subhanpura,54,54,54,54,54,54


**Let's find out how many unique categories can be curated from all the returned venues**

In [31]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 58 uniques categories.


In [32]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Ice Cream Shop', 'Pizza Place', 'Asian Restaurant',
       'Indian Restaurant', 'Fast Food Restaurant', 'Dessert Shop',
       'Hotel Bar', 'Snack Place', 'Hotel', 'Shopping Mall', 'Café',
       'Donut Shop', 'Sandwich Place', 'Multiplex', 'Sports Bar', 'Gym',
       "Women's Store", 'Smoke Shop', 'Bakery', 'Bookstore',
       'Italian Restaurant', 'BBQ Joint', 'Mexican Restaurant',
       'Chinese Restaurant', 'Movie Theater', 'Arts & Crafts Store',
       'Salad Place', 'Tea Room', 'Food Court', 'Miscellaneous Shop',
       'Mediterranean Restaurant', 'Department Store', 'Clothing Store',
       'Bowling Alley', 'Bus Station', 'Coffee Shop', 'Cosmetics Shop',
       'Juice Bar', 'Stadium', 'Food Truck', 'Train Station', 'Garden',
       'Airport', 'Airport Service', 'Airport Lounge', 'Platform',
       'Moving Target', 'Garden Center', 'Pie Shop', 'Convention Center'],
      dtype=object)

In [33]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

False

### 6. Analyze Each Neighborhood

In [34]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(506, 59)


,Neighborhoods,Airport,Airport Lounge,Airport Service,Arts & Crafts Store,Asian Restaurant,Awadhi Restaurant,BBQ Joint,Bakery,Bookstore,Bowling Alley,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convention Center,Cosmetics Shop,Department Store,Dessert Shop,Donut Shop,Fast Food Restaurant,Food Court,Food Truck,Garden,Garden Center,Golf Course,Gym,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Market,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Multiplex,Pie Shop,Pizza Place,Platform,Restaurant,Salad Place,Sandwich Place,Shopping Mall,Smoke Shop,Snack Place,Sports Bar,Stadium,Tea Room,Train Station,Women's Store
0,Akota,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Akota,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Akota,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Akota,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Akota,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [35]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(11, 59)


,Neighborhoods,Airport,Airport Lounge,Airport Service,Arts & Crafts Store,Asian Restaurant,Awadhi Restaurant,BBQ Joint,Bakery,Bookstore,Bowling Alley,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convention Center,Cosmetics Shop,Department Store,Dessert Shop,Donut Shop,Fast Food Restaurant,Food Court,Food Truck,Garden,Garden Center,Golf Course,Gym,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Market,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Multiplex,Pie Shop,Pizza Place,Platform,Restaurant,Salad Place,Sandwich Place,Shopping Mall,Smoke Shop,Snack Place,Sports Bar,Stadium,Tea Room,Train Station,Women's Store
0,Akota,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.025000,0.025000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,0.000000,0.000000,0.025000,0.025,0.025000,0.275000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,0.000000,0.100000,0.000000,0.000000,0.000000,0.050000,0.025000,0.025,0.025000,0.025000,0.000000,0.000000,0.000000,0.025000
1,Alkapuri,0.000000,0.000000,0.000000,0.010870,0.021739,0.000000,0.032609,0.000000,0.010870,0.010870,0.010870,0.130435,0.021739,0.021739,0.010870,0.000000,0.01087,0.010870,0.010870,0.000000,0.076087,0.010870,0.000000,0.000000,0.000000,0.000000,0.010870,0.000000,0.000000,0.032609,0.000,0.010870,0.130435,0.043478,0.000000,0.021739,0.000000,0.000000,0.010870,0.021739,0.010870,0.010870,0.000000,0.076087,0.000000,0.032609,0.000000,0.000000,0.010870,0.032609,0.065217,0.000,0.032609,0.010870,0.010870,0.021739,0.000000,0.000000
2,Fatehgunj,0.027027,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.054054,0.027027,0.000000,0.054054,0.135135,0.027027,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.135135,0.027027,0.027027,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000,0.027027,0.081081,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.027027,0.000000,0.027027,0.000000,0.027027,0.000000,0.000000,0.000000,0.027027,0.054054,0.000,0.081081,0.000000,0.000000,0.000000,0.027027,0.000000
3,Karelibaug,0.032258,0.032258,0.064516,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.129032,0.032258,0.000000,0.032258,0.000000,0.00000,0.000000,0.000000,0.000000,0.096774,0.000000,0.032258,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.032258,0.000,0.032258,0.032258,0.000000,0.000000,0.096774,0.000000,0.000000,0.000000,0.032258,0.000000,0.032258,0.000000,0.032258,0.000000,0.032258,0.000000,0.000000,0.000000,0.064516,0.032258,0.000,0.032258,0.000000,0.000000,0.032258,0.000000,0.000000
4,Manjalpur,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.111111,0.000000,0.00000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.111111,0.000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000
5,"Race Course, Vadodara",0.000000,0.000000,0.000000,0.013699,0.013699,0.000000,0.041096,0.000000,0.013699,0.013699,0.013699,0.082192,0.013699,0.027397,0.000000,0.013699,0.00000,0.013699,0.013699,0.013699,0.068493,0.000000,0.013699,0.000000,0.013699,0.000000,0.027397,0.000000,0.000000,0.013699,0.000,0.013699,0.150685,0.054795,0.000000,0.000000,0.000000,0.000000,0.013699,0.013699,0.013699,0.000000,0.000000,0.082192,0.013699,0.041096,0.000000,0.000000,0.013699,0.027397,0.068493,0.000,0.027397,0.013699,0.013699,0.013699,0.000000,0.000000
6,Raopura,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.

In [36]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

10

**Create a new DataFrame for Shopping Mall data only**

In [37]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [38]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Akota,0.025000
1,Alkapuri,0.065217
2,Fatehgunj,0.054054
3,Karelibaug,0.032258
4,Manjalpur,0.111111


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Vadodara into 3 clusters.

In [39]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 2, 1, 0, 2, 0, 0, 1])

In [40]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [41]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Akota,0.025000,2
1,Alkapuri,0.065217,0
2,Fatehgunj,0.054054,0
3,Karelibaug,0.032258,2
4,Manjalpur,0.111111,1


In [42]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(11, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Akota,0.025000,2,22.29128,73.16947
1,Alkapuri,0.065217,0,22.31419,73.17424
2,Fatehgunj,0.054054,0,22.32548,73.18949
3,Karelibaug,0.032258,2,22.32567,73.20176
4,Manjalpur,0.111111,1,22.27547,73.19164


In [43]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(11, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
1,Alkapuri,0.065217,0,22.31419,73.17424
2,Fatehgunj,0.054054,0,22.32548,73.18949
5,"Race Course, Vadodara",0.068493,0,22.31234,73.16282
7,Sayajigunj,0.074627,0,22.31281,73.18767
8,Subhanpura,0.092593,0,22.32367,73.16472
10,Template:Urban and suburban areas of Vadodara,0.069444,0,22.30946,73.17993
4,Manjalpur,0.111111,1,22.27547,73.19164
9,Tandalja,0.142857,1,22.28469,73.14562
0,Akota,0.025000,2,22.29128,73.16947
3,Karelibaug,0.032258,2,22.32567,73.20176


**Finally, let's visualize the resulting clusters**

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [45]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [46]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
1,Alkapuri,0.065217,0,22.31419,73.17424
2,Fatehgunj,0.054054,0,22.32548,73.18949
5,"Race Course, Vadodara",0.068493,0,22.31234,73.16282
7,Sayajigunj,0.074627,0,22.31281,73.18767
8,Subhanpura,0.092593,0,22.32367,73.16472
10,Template:Urban and suburban areas of Vadodara,0.069444,0,22.30946,73.17993


#### Cluster 1

In [47]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
4,Manjalpur,0.111111,1,22.27547,73.19164
9,Tandalja,0.142857,1,22.28469,73.14562


#### Cluster 2

In [48]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Akota,0.025000,2,22.29128,73.16947
3,Karelibaug,0.032258,2,22.32567,73.20176
6,Raopura,0.000000,2,22.30394,73.19981


#### Observations:
Most of the shopping malls are concentrated in the central area of Vadodara city, with the highest number in cluster 0 and moderate number in cluster 2. On the other hand, cluster 1 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 0 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. 